In [2]:
import itertools
import matplotlib.pyplot as plt
import pathlib
import random
import torch
import torch.nn
import torch.nn.functional
import tqdm

In [13]:
# Defaults
DIR_READ = pathlib.Path(__name__).resolve().parent
DIR_OUT = pathlib.Path(__name__).resolve().parents[1] / "out" / "makemore-mlp"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
g = torch.Generator(device=device).manual_seed(2147483647) # default seed from the torch docs

In [4]:
# reading data to tokens
def load_txt_to_list(fname):
    path = pathlib.Path(fname)
    if not path.exists():
        raise f"{fname} not found!"

    list_out = []
    with open(path, "r") as file:
        list_out = file.read().splitlines()

    return list_out

def construct_map_token_to_index(list_tokens):
    """
    Returns { token : index }

    Map each token in the vocab to a unique integer,
    which will be its index into the Bag of words vector

    TODO: https://en.wikipedia.org/wiki/Feature_hashing
    NOTE: Fatal flaw is set sorting is random, making debugging a little harder
    """
    dict_to_ix = {}
    dict_to_word = {}
    for i, token in enumerate(set(list_tokens)):
        dict_to_ix[token] = i
        dict_to_word[i] = token

    return dict_to_ix, dict_to_word

fname = DIR_READ / "names.txt"
words = load_txt_to_list(fname)

dict_token_to_ix, dict_ix_to_token = construct_map_token_to_index("".join(words))

list_tokens_extra = ["."]
for token in list_tokens_extra:
    dict_token_to_ix[token] = len(dict_token_to_ix)
    dict_ix_to_token[len(dict_ix_to_token)] = token

In [14]:
# Calculating the ngrams
list_documents = [ list(string) + ["."] for string in words ] # different from before

def construct_vector_ngram(tokens_context, dict_index):
    return list(map(lambda w: dict_index[w], tokens_context))

def construct_n_grams(list_documents, dict_index, size_context, size_prediction=1):
    """
    Initialize an empty window
    Update as you go along

    NOTE: Creates size_context-1 more vectors to account for the stop tokens
    e.g., size_context = 3-1 creates
        [<S>, <S>, <T>]

    TODO: Account for when size_prediction is larger than length(document)
    """
    list_out = []
    vector = [dict_index["."]]*size_context
    for tokens in list_documents:
        # TODO: Right pad tokens to size_prediction
        for i in range(0, len(tokens)):
            vector = vector[size_prediction:] + construct_vector_ngram(tokens[i:i+size_prediction], dict_index)
            list_out.append(vector)
    return list_out

SIZE_CONTEXT = 4 # SIZE_NGRAMS+1 or BLOCKSIZE
list_vectors = construct_n_grams(list_documents, dict_token_to_ix, size_context=SIZE_CONTEXT)
# Verify the mapping
for document in list_documents[:3]:
    for i in range(len(document)): # num_ngrams
        print((["."]*(SIZE_CONTEXT-1) + document)[i:i+SIZE_CONTEXT], list_vectors[i])

matrix_ngrams = torch.tensor(list_vectors, dtype = torch.int64, device=device)

['.', '.', '.', 'e'] [26, 26, 26, 6]
['.', '.', 'e', 'm'] [26, 26, 6, 21]
['.', 'e', 'm', 'm'] [26, 6, 21, 21]
['e', 'm', 'm', 'a'] [6, 21, 21, 8]
['m', 'm', 'a', '.'] [21, 21, 8, 26]
['.', '.', '.', 'o'] [26, 26, 26, 6]
['.', '.', 'o', 'l'] [26, 26, 6, 21]
['.', 'o', 'l', 'i'] [26, 6, 21, 21]
['o', 'l', 'i', 'v'] [6, 21, 21, 8]
['l', 'i', 'v', 'i'] [21, 21, 8, 26]
['i', 'v', 'i', 'a'] [21, 8, 26, 23]
['v', 'i', 'a', '.'] [8, 26, 23, 15]
['.', '.', '.', 'a'] [26, 26, 26, 6]
['.', '.', 'a', 'v'] [26, 26, 6, 21]
['.', 'a', 'v', 'a'] [26, 6, 21, 21]
['a', 'v', 'a', '.'] [6, 21, 21, 8]


# E01: Hyperparameter Search
Tune the hyperparameters of the training to beat my best validation loss of 2.2

Author's Note: I KNOW this can be done easier in pytorch & sklearn, but it's better to see how this works for educational reasons.

In [6]:
def forward_pass(X, Y, list_parameters):
    """
    logits = tanh \circ (C[X] @ W1 + b1)) @ W2 + b2
    loss = cross_entropy(logits, Y)
    where 
        X is the tokenized inputs, Y is the tokenized outputs
        C is the (|V|, k) mapping that embeds X into a lower dimensional space
        W1 is the (k, hidden) weight matrix
        b1 is the (hidden, 1) bias vector
        W2 is the (hidden, |V|) weight matrix
        b2 = is the (|V|) bias vector
    
    i.e,
        logits = linear \circ tanh \circ linear \circ embedding
    """
    embed = list_parameters[0][X] # shape (num_ngrams[indices], SIZE_CONTEXT-1, SIZE_DIMENSION)
    h = torch.tanh(torch.einsum('ijk,jkl -> il', embed, list_parameters[1]) + list_parameters[2]) # hidden states
    logits = h @ list_parameters[3] + list_parameters[4]
    loss = torch.nn.functional.cross_entropy(logits, Y)

    return loss

def train_neural_net(xtr, ytr, list_parameters, num_epochs = 30000, size_batch = 35, tag = None):
    list_steps = []
    list_loss = []

    LEARNING_RATE = 0.1 # discovered empirocally
    epoch_decay = num_epochs*0.9 # after 90% of epochs
    for p in list_parameters:
        p.requires_grad=True
    for i in range(num_epochs):
        # mini-batch
        indices = torch.randint(0, xtr.shape[0], (size_batch,))
        loss = forward_pass(xtr[indices], ytr[indices], list_parameters)
        # backward pass
        for p in list_parameters:
            p.grad = None
        loss.backward()

        # update parameters
        if i == epoch_decay:
            print("\tLoss before learning rate decay:", loss.item())
            LEARNING_RATE = LEARNING_RATE * 0.1
        for p in list_parameters:
            p.data += -LEARNING_RATE * p.grad # going against the gradient reduces the loss
        list_steps.append(i)
        list_loss.append(loss.log10().item())

    print("\tLoss after learning rate decay:", loss.item())
    plt.title(f"Train log10-loss {tag}.png")
    plt.plot(list_steps, list_loss)
    plt.savefig(DIR_OUT / f"train-log10-loss-{sum(p.numel() for p in list_parameters)}-{tag}.png")
    plt.close()

    return list_parameters

In [15]:
# Train/Test Split
random.seed(42)
random.shuffle(matrix_ngrams.clone())
xs = matrix_ngrams[:,0:SIZE_CONTEXT-1]
ys = matrix_ngrams[:,-1] # vector

n1 = int(0.8 * xs.shape[0])
n2 = int(0.9 * xs.shape[0])
Xtr, Xdev, Xts = xs.tensor_split((n1, n2), dim=0)
Ytr, Ydev, Yts = ys.tensor_split((n1, n2), dim=0)

In [22]:
dict_hyperparameters = {
    "DIMENSIONS": [4,6,8,10,12],
    "HIDDENS": [100, 200, 300, 400, 500]
}
SIZE_BATCH = 128
BEST_PARAMETERS = None
BEST_LOSS = float('inf')
list_list_parameters = []
num_epochs = 10000
print("HYPERPARAMETER-TRAINING")
for (SIZE_DIMENSION, SIZE_HIDDEN) in tqdm.tqdm(list(itertools.product(*list(dict_hyperparameters.values())))):
    C = torch.randn(size=(len(dict_ix_to_token),SIZE_DIMENSION), generator=g, device=device)
    W1 = torch.randn(size=(SIZE_CONTEXT-1, SIZE_DIMENSION, SIZE_HIDDEN), generator=g, device=device)
    b1 = torch.randn(SIZE_HIDDEN, generator=g, device=device)
    W2 = torch.randn(size=(SIZE_HIDDEN, len(dict_ix_to_token)), generator=g, device=device)
    b2 = torch.randn(len(dict_ix_to_token), generator=g, device=device)
    list_parameters = [C, W1, b1, W2, b2]

    tag = (SIZE_DIMENSION, SIZE_HIDDEN)
    print(f"TRAINING NEURAL NET with (DIMENSION={SIZE_DIMENSION}, HIDDEN={SIZE_HIDDEN})")
    list_parameters_updated = train_neural_net(Xtr, Ytr, list_parameters, num_epochs, SIZE_BATCH, tag)
    loss_train = forward_pass(Xtr, Ytr, list_parameters_updated)
    loss_dev = forward_pass(Xdev, Ydev, list_parameters_updated)
    print("\tTraining loss:", loss_train.item())
    print("\tValidate loss:", loss_dev.item())
    if loss_dev.item() < BEST_LOSS:
        print(f"\tBest loss updated for (DIMENSION, HIDDEN): {SIZE_DIMENSION, SIZE_HIDDEN}")
        BEST_LOSS = loss_dev.item()
        BEST_PARAMETERS = (SIZE_DIMENSION, SIZE_HIDDEN)

print("\tBest loss",BEST_LOSS, "with (DIMENSION, HIDDEN)", BEST_PARAMETERS)

HYPERPARAMETER-TRAINING


  0%|          | 0/25 [00:00<?, ?it/s]

TRAINING NEURAL NET with (DIMENSION=4, HIDDEN=100)
	Loss before learning rate decay: 2.3964381217956543
	Loss after learning rate decay: 2.3666436672210693


  4%|▍         | 1/25 [00:49<19:48, 49.51s/it]

	Training loss: 2.322718620300293
	Validate loss: 2.542935371398926
	Best loss updated for (DIMENSION, HIDDEN): (4, 100)
TRAINING NEURAL NET with (DIMENSION=4, HIDDEN=200)
	Loss before learning rate decay: 2.39103102684021
	Loss after learning rate decay: 2.2972142696380615


...

	Training loss: 2.2575223445892334
	Validate loss: 2.587883234024048
TRAINING NEURAL NET with (DIMENSION=12, HIDDEN=400)
	Loss before learning rate decay: 2.448808431625366
	Loss after learning rate decay: 2.54472017288208


 96%|█████████▌| 24/25 [27:11<01:19, 79.54s/it]

	Training loss: 2.2624053955078125
	Validate loss: 2.6233489513397217
TRAINING NEURAL NET with (DIMENSION=12, HIDDEN=500)
	Loss before learning rate decay: 2.697355031967163
	Loss after learning rate decay: 2.204313039779663


100%|██████████| 25/25 [28:11<00:00, 67.66s/it]

	Training loss: 2.2610063552856445
	Validate loss: 2.6454756259918213
	Best loss 2.4923994541168213 with (DIMENSION, HIDDEN) (12, 200)


# E02: Initialize Weights More Carefully
I was not careful with the intialization of the network in this video. 

(1) What is the loss you'd get if the predicted probabilities at initialization were perfectly uniform? What loss do we achieve? 

(2) Can you tune the initialization to get a starting loss that is much more similar to (1)?

The tensors were mistakenly initialized using randn instead of rand, which samples from a standard normal distribution 

# E03: Read the Bengio et al 2003 paper (link above), implement and try any idea from the paper. 
Did it work?
